In [ ]:
# !apt update
# !apt install chromium-chromedriver
# !pip install -U selenium
# ! pip install -qU selenium webdriver-manager persiantools

In [28]:
import re
from selenium import webdriver
# from selenium.webdriver.remote.webelement import WebElement

In [30]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36")

driver = webdriver.Chrome("chromedriver", options=options)

In [6]:
query = "دورخهچ"

In [7]:
driver.get("https://www.google.com/search?q=" + query)

In [8]:
# # save page source
# page_source = driver.page_source
# # save page source to file
# with open("page_source.html", "w") as f:
#     f.write(page_source)

In [9]:
# xpath=//div[@id='rcnt']
text = driver.find_elements("xpath", "//div[contains(@class, 'BmP5tf')]")
for t in text:
    print(t.text)

Showing results for دوچرخه
Search instead for دورخهچ
Next >


In [4]:
import json
from tqdm import tqdm

import numpy as np
import pandas as pd

In [5]:
class JsonFileIterator:
    def __init__(self, path):
        self.path = path
        self.f = open(path, "r")
        self.i = 0
        self.length = self.counter_lines()

    def __iter__(self):
        return self

    def __next__(self):
        line = self.f.readline()
        if not line:
            # End of file
            self.f.close()
            raise StopIteration
        self.i += 1
        return json.loads(line)

    def counter_lines(self):
        with open(self.path, "r") as f1:
            return sum(1 for _ in f1)

    def __len__(self):
        return self.length


In [6]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.

    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

# search_query = pd.DataFrame(read_json_lines('./torob/torob-search-data_v1.jsonl'))
search_query = pd.DataFrame(read_json_lines('./torob/torob-search-data_v1.jsonl', n_lines=200000))
search_query.head()

,raw_query,result,clicked_result,clicked_rank,timestamp
0,لوستر سقفی برنز,"[7151290, 6462477, 7385791, 8451497, None, 269...",[9457219],[16],2022-07-24T09:21:58.752000+00:00
1,قیمت هلیکوپتر,"[363737, 3147253, 8720128, 9796388, 1420685, 5...","[3147253, 7135387]","[1, 7]",2022-07-24T07:32:12.261000+00:00
2,ساعت هوشمند,"[2459592, 9391819, 4229448, 7824893, 1670767, ...","[900897, 2931230, 31302]","[7, 25, 52]",2022-07-24T02:51:35.643000+00:00
3,تفلون مایع,"[5428407, 4474271, 4581189, 8504749, 4131340, ...","[7660686, 5901997, 2376830, 2383125]","[19, 18, 21, 36]",2022-07-24T02:51:54.771000+00:00
4,خط زن,"[8466568, 9285461, 1105044, 1314054, 1791930, ...",[3852139],[20],2022-07-23T20:56:36.043000+00:00


In [ ]:
## get unique queries
# unique_queries = search_query['raw_query'].unique()

In [7]:
queries = dict()
for search in tqdm(search_query.to_dict('records')):
    query = search["raw_query"]
    if query not in queries:
        queries[query] = 1
    else:
        queries[query] += 1

100%|██████████| 200000/200000 [00:00<00:00, 1265788.47it/s]


In [8]:
sorted_queries = sorted(queries.items(), key=lambda x: x[1], reverse=False)

In [10]:
df = pd.DataFrame.from_dict(queries, orient='index', columns=['count'])
df.sort_values(by=['count'], inplace=True, ascending=True)

In [23]:
temp = df.copy(deep=True)

In [24]:
# drop count more than 1
temp = temp[temp['count'] == 1]

In [31]:
# number of queries with count 1
# df[df['count'] == 1].shape[0]
temp

,count
قفل دیسکی,1
ساعت ضد اب زنانه,1
cute,1
کبف مدرسه پسرانه,1
خانه هوشمند,1
...,...
وایتبرد,1
فلش ۱۶ گیگابایت,1
قهوه عربیکا,1
لباسشویی پاکشوما 8,1


In [40]:
import csv
import logging

In [ ]:
# with open("retrive.txt", "w", encoding="utf-8", newline="") as csvfile:

#     wrtiter = csv.writer(csvfile)
#     wrtiter.writerow(
#         [
#             "query",
#             "spell_checked",
#             "title_text",
#         ]
#     )

#     for query in tqdm(temp.index):

#         driver.get("https://torob.com/search/?query=" + query)
        
#         # print(f"Query>>>>>> {query} <<<<<<")

#         check_spell = driver.find_elements("xpath", "//p[contains(@class, 'black_p')]")

#         torob_spell_check = False
#         corrected_form = None
#         for text in check_spell:
#             extracted_text = text.get_attribute("innerText")
#             if extracted_text.startswith("جستجو غلط‌گیری شد:"):
#                 torob_spell_check = True
#                 # logging.warning(f"\033[92m{extracted_text}\033[0m")
#                 corrected_query = re.search(r"جستجو غلط‌گیری شد: (.*)", extracted_text).group(1)
#                 # logging.warning(f"\033[92m Corrected Query: {corrected_query}\033[0m")
#                 corrected_form = corrected_query
#             else:
#                 corrected_form = None

#         names = driver.find_elements("xpath", "//h2[contains(@class,'jsx-fa8eb4b3b47a1d18')]")
#         name_list = []
#         for name in names:
#             extracted_name = name.get_attribute("innerText")
#             # print(extracted_name)
#             name_list.append(extracted_name)

#         title_text = '\n'.join(name_list)

#         # retrive_dict = dict()
#         # retrive_dict['query'] = query
#         # retrive_dict['spell_check'] = corrected_form
#         # retrive_dict['products_text'] = title_text
    
#         wrtiter.writerow(
#             [
#                 query,
#                 corrected_form,
#                 title_text,
#             ]
#         )

#     retrive_list.append(retrive_dict)

In [53]:
df = pd.read_csv('./spell_check/output_4.csv', index_col=0)
temp = df.copy(deep=True)

In [ ]:
temp

In [55]:
with open("retrive_part_5.txt", "w", encoding="utf-8", newline="") as csvfile:

    wrtiter = csv.writer(csvfile)
    wrtiter.writerow(
        [
            "query",
            "spell_checked",
            "title_text",
        ]
    )

    for query in tqdm(temp.index):

        driver.get("https://torob.com/search/?query=" + query)

        check_spell = driver.find_elements("xpath", "//p[contains(@class, 'black_p')]")

        torob_spell_check = False
        corrected_form = None
        for text in check_spell:
            extracted_text = text.get_attribute("innerText")
            if extracted_text.startswith("جستجو غلط‌گیری شد:"):
                torob_spell_check = True
                corrected_query = re.search(
                    r"جستجو غلط‌گیری شد: (.*)", extracted_text
                ).group(1)
                corrected_form = corrected_query
            else:
                corrected_form = None

        names = driver.find_elements(
            "xpath", "//h2[contains(@class,'jsx-fa8eb4b3b47a1d18')]"
        )
        name_list = []
        for name in names:
            extracted_name = name.get_attribute("innerText")
            name_list.append(extracted_name)

        title_text = "\n".join(name_list)

        wrtiter.writerow(
            [
                query,
                corrected_form,
                title_text,
            ]
        )


  0%|          | 156/48357 [06:58<35:53:15,  2.68s/it] 


KeyboardInterrupt: 

In [43]:
df = pd.read_csv('retrive_part_5.txt')

In [47]:
print(df.iloc[0].title_text)

قفل دیسکی آژیر دار CAP ا قفل دیسک موتور آژیردار
قفل دیسکی ردلاین مدل 064 مناسب انواع موتور سیکلت
قفل دیسکی ردلاین مدل 074 مناسب انواع موتور سیکلت شهری
قفل دیسک ردلاین مدل RE210 ا قفل دیسکی موتور سنگین ردلاین
قفل دیسکی ردلاین مدل RE420
قفل دیسکی موتور سیکلت آلفا
قفل دیسکی موتورسیکلت مدل abous
قفل کتابی کهن کلید دیسکی
دوشاخ جلو دوچرخه برند Olympia کمک فنر جلو دوچرخه مدل قفل کن دار تنظیمی سایز 27.5 قابلیت دیسکی و ویبریک Olympia Fork Bicycle Size 27.5
دوشاخ جلو دوچرخه برند Olympia مدل قفل کن دار تنظیمی سایز 29 قابلیت دیسکی و ویبریک Olympia Fork Bicycle Size 29
دوشاخ دوچرخه کوهستان برند OLYMPIA قفل شو سایز 29 دیسکی و ویبریک {کمک فنر جلو دوچرخه }OLYMPIA MTB Bicycle Fork Size 29
قفل دیسکی ویندی مدل boot
قفل دیسکی موتورسیکلت آلفا / ALFA
قفل دیسکی ردلاین مدل RE420
قفل دیسکی ردلاین مدل RE420 سبز فسفری
قفل دیسکی موتور سیکلت آنلاین مدل02RD
قفل دیسکی موتور سیکلت آنلاین مدل02BK
قفل دیسکی موتور سیکلت آنلاین مدل02BL
قفل دیسکی موتور سیکلت ویژن مدل DX21-BLK آژیر دار
قفل دیسکی ردلاین مدل RE420
قفل دیسکی 

In [39]:
retrive_list

[{'query': 'قفل دیسکی',
  'spell_check': None,
  'products_text': 'قفل دیسکی آژیر دار CAP ا قفل دیسک موتور آژیردار\nقفل دیسکی ردلاین مدل 064 مناسب انواع موتور سیکلت\nقفل دیسکی ردلاین مدل 074 مناسب انواع موتور سیکلت شهری\nقفل دیسک ردلاین مدل RE210 ا قفل دیسکی موتور سنگین ردلاین\nقفل دیسکی ردلاین مدل RE420\nقفل دیسکی موتور سیکلت آلفا\nقفل دیسکی موتورسیکلت مدل abous\nقفل کتابی کهن کلید دیسکی\nدوشاخ جلو دوچرخه برند Olympia کمک فنر جلو دوچرخه مدل قفل کن دار تنظیمی سایز 27.5 قابلیت دیسکی و ویبریک Olympia Fork Bicycle Size 27.5\nدوشاخ جلو دوچرخه برند Olympia مدل قفل کن دار تنظیمی سایز 29 قابلیت دیسکی و ویبریک Olympia Fork Bicycle Size 29\nدوشاخ دوچرخه کوهستان برند OLYMPIA قفل شو سایز 29 دیسکی و ویبریک {کمک فنر جلو دوچرخه }OLYMPIA MTB Bicycle Fork Size 29\nقفل دیسکی ویندی مدل boot\nقفل دیسکی موتورسیکلت آلفا / ALFA\nقفل دیسکی ردلاین مدل RE420\nقفل دیسکی ردلاین مدل RE420 سبز فسفری\nقفل دیسکی موتور سیکلت آنلاین مدل02RD\nقفل دیسکی موتور سیکلت آنلاین مدل02BK\nقفل دیسکی موتور سیکلت آنلاین مدل02BL\nق

In [ ]:
# %%reset -f

In [ ]:
search_data = JsonFileIterator("./torob/test-offline-data_v1.jsonl")
queries = dict()
for search in tqdm(search_data):
    query = search["raw_query"]
    if query not in queries:
        queries[query] = 1
    else:
        queries[query] += 1
# queries = sorted(queries.items(), key=lambda x: x[1], reverse=True)

In [ ]:
queries

In [ ]:
# cocnvert to numpy array
import numpy as np

In [ ]:
queries = np.array(list(queries.items()))

In [ ]:
query = "وشی نوت 10  سامسونگ"

In [ ]:
driver.get("https://torob.com/search/?query=" + query)

In [ ]:
import logging
logging.warning('Noice!')

In [ ]:
driver.get("https://google.com/search?q=" + query)

In [ ]:
check_spell = driver.find_elements("xpath", "//p[contains(@class, 'black_p')]")
torob_spell_check = False
corrected_form = None
for text in check_spell:
    extracted_text = text.get_attribute("innerText")
    if extracted_text.startswith("جستجو غلط‌گیری شد:"):
        torob_spell_check = True
        logging.warning(f"\033[92m{extracted_text}\033[0m")
        corrected_query = re.search(r"جستجو غلط‌گیری شد: (.*)", extracted_text).group(1)
        logging.warning(f"\033[92m Corrected Query: {corrected_query}\033[0m")
        corrected_form = corrected_query

names = driver.find_elements("xpath", "//h2[contains(@class,'jsx-fa8eb4b3b47a1d18')]")
name_list = []
for name in names:
    extracted_name = name.get_attribute("innerText")
    print(extracted_name)
    name_list.append(extracted_name)

# print in red
print(f"\033[91m{'='*50}\033[0m")
print(f"length of name_list: {len(name_list)}")
print(f"\033[91m{'='*50}\033[0m")

In [ ]:
print(query)
query_in_results = 0
for name in name_list:
    if query in name:
        query_in_results += 1

if torob_spell_check or query_in_results < 5:
    print(f"query «{query}» is not in at least 5 of the results")

In [ ]:
bi_gram_words = set()
for title in name_list:
    list_of_words = title.split()
    for i in range(len(list_of_words) - 1):
        bi_gram_words.add(list_of_words[i] + " " + list_of_words[i + 1])

bi_gram_words

In [ ]:
all_words = set([word for name in name_list for word in name.split()])
all_words = all_words.union(bi_gram_words)


In [ ]:
# from parsivar import SpellCheck
# spell_checker = SpellCheck()
# corrected_text = spell_checker.spell_corrector("your text here")

In [ ]:
def MED(sent_01, sent_02):
    n = len(sent_01)
    m = len(sent_02)

    matrix = [[i + j for j in range(m + 1)] for i in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if sent_01[i - 1] == sent_02[j - 1]:
                d = 0
            else:
                d = 1

            matrix[i][j] = min(
                matrix[i - 1][j] + 1, matrix[i][j - 1] + 1, matrix[i - 1][j - 1] + d
            )

    distance_score = matrix[n][m]

    return distance_score

In [ ]:
candidate_words = []
for word in all_words:
    if MED(query, word) <= 3:
        candidate_words.append((word, MED(query, word)))

candidate_words.sort(key=lambda x: x[1])
candidate_words

In [ ]:
query_in_results = sum([1 for name in name_list if query in name])
query_in_results

In [ ]:
# minimum edit distance
